<a href="https://colab.research.google.com/github/xiaomeng-ma/ParentBERTo/blob/master/AllChildren_WordEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus.reader import CHILDESCorpusReader
import numpy as np
import pandas as pd
from nltk.probability import FreqDist
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

##Getting All Parents' Data

In [ ]:
corpus = "/content/drive/My Drive/corpus data/Data/US-Data"

In [ ]:
Data = CHILDESCorpusReader(corpus, '.*.xml')

In [ ]:
all_data = []
for f in Data.fileids():
  sents = Data.sents(f, speaker = ['MOT', 'FAT'])
  word2sent = []
  for sent in sents:
    strsent = " ".join(i for i in sent)
    word2sent.append(strsent)
  sents_in_file = "\n".join(word2sent)
  all_data.append([f, sents_in_file])

In [ ]:
df=pd.DataFrame(all_data, columns=['file','parent sentence'])

In [ ]:
df.to_csv('All Parents Sentence')
!cp 'All Parents Sentence' "/content/drive/My Drive/Dissertation"

##Train a word embedding

In [2]:
url = "https://raw.githubusercontent.com/xiaomeng-ma/CHILDES/master/All%20Parents%20Sentence.csv"
df = pd.read_csv(url)

In [3]:
df= df.dropna()

In [ ]:
len(df['parent sentence'])


5150

In [5]:
df['parent sentence']

0       what's that\nit's a chicken\nyeah\nwhat's this...
1       wanna stack them for me\n\n\n\noh\ndid you say...
2       try another one\nthose are very hard\nlet me c...
3       you like to open the doors of the house\nthat'...
4       dog\nuhoh\ndog\n dog\ntwo dogs\ntwo more ladie...
                              ...                        
6902    what are you doing\nyour gonna find the eggs\n...
6903    I thought it was a spider\nit looks like a spi...
6904    it looks like uh Harry Potter or something\nis...
6905    where\nyeah what what is this\nno what\nis it ...
6906    he doesn't watch the Smurfs or anything\nI jus...
Name: parent sentence, Length: 5150, dtype: object

In [10]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t',
    lower=True, split='\n', char_level=False, oov_token=None)
toked = tokenizer.fit_on_texts(df['parent sentence'])

In [11]:
len(tokenizer.word_index) + 1

666383

In [ ]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.trainers import BpeTrainer

tokenizer = ByteLevelBPETokenizer()
trainer = BpeTrainer(vocab_size = 52_000, min_frequency = 2, special_tokens = [  
    "<pad>",
    "<s>",
    "</s>",
    "<unk>",
    "<mask>",])
tokenizer.train_from_iterator(df['parent sentence'],vocab_size = 52_000, min_frequency = 2, special_tokens = [  
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",])

In [ ]:
#!mkdir "/content/drive/My Drive/Dissertation/ParentBERTo"
dir = "/content/drive/My Drive/Dissertation/ParentBERTo"
tokenizer.save_model(dir)

['/content/drive/My Drive/Dissertation/ParentBERTo/vocab.json',
 '/content/drive/My Drive/Dissertation/ParentBERTo/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
tokenizer = ByteLevelBPETokenizer(
    '/content/drive/My Drive/Dissertation/ParentBERTo/vocab.json',
 '/content/drive/My Drive/Dissertation/ParentBERTo/merges.txt',
)
tokenizer._tokenizer.post_processor= BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_padding(direction = "right", pad_id = 1, pad_type_id = 0, pad_token = "<pad>")

In [ ]:
encode = tokenizer.encode("I think he's coming")
print(encode.tokens)
print(encode.ids)

['<s>', 'I', 'Ġthink', 'Ġhe', "'s", 'Ġcoming', '</s>']
[0, 45, 399, 301, 279, 1315, 2]


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=128,
    num_attention_heads=8,
    num_hidden_layers=4,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(dir, max_len=200)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print("Number of parameters:", model.num_parameters())
print(model)

In [ ]:
with open("/content/drive/My Drive/Dissertation/Parent-Sents.txt", "w") as f:
  for text in df['parent sentence'].tolist():
    f.write(text + '\n')

In [ ]:
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
tokenizer = tokenizer,
file_path = "/content/drive/My Drive/Dissertation/Parent-Sents.txt",
block_size = 128,
    
)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=dir,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,7.066000
1000,6.007100
1500,5.597000
2000,5.261100
2500,5.041900
3000,4.936900
3500,4.804900
4000,4.720600
4500,4.602800
5000,4.518100


TrainOutput(global_step=36478, training_loss=3.9929319321373926, metrics={'train_runtime': 4039.2625, 'train_samples_per_second': 9.031, 'total_flos': 9504757263348096.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 327290, 'init_mem_gpu_alloc_delta': 276130304, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 773927, 'train_mem_gpu_alloc_delta': 835963904, 'train_mem_cpu_peaked_delta': 202714820, 'train_mem_gpu_peaked_delta': 3461976064})

In [ ]:
trainer.save_model(dir)

##Language Model Results

In [ ]:
from transformers import pipeline
fill_mask = pipeline(
    'fill-mask',
    model = dir,
    tokenizer = dir
)

Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/My Drive/Dissertation/ParentBERTo and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
fill_mask("I <mask> you")

[{'score': 0.4584316611289978,
  'sequence': 'I see you',
  'token': 384,
  'token_str': ' see'},
 {'score': 0.049191635102033615,
  'sequence': 'I love you',
  'token': 1041,
  'token_str': ' love'},
 {'score': 0.03835704177618027,
  'sequence': 'I got you',
  'token': 449,
  'token_str': ' got'},
 {'score': 0.03673850744962692,
  'sequence': 'I know you',
  'token': 378,
  'token_str': ' know'},
 {'score': 0.029599659144878387,
  'sequence': 'I told you',
  'token': 1382,
  'token_str': ' told'}]

In [ ]:
fill_mask("I pick <mask> up")

[{'score': 0.3976026773452759,
  'sequence': 'I pick it up',
  'token': 290,
  'token_str': ' it'},
 {'score': 0.11565425246953964,
  'sequence': 'I pick you up',
  'token': 273,
  'token_str': ' you'},
 {'score': 0.08241920918226242,
  'sequence': 'I pick them up',
  'token': 495,
  'token_str': ' them'},
 {'score': 0.08196157962083817,
  'sequence': 'I pick  up',
  'token': 225,
  'token_str': ' '},
 {'score': 0.039304427802562714,
  'sequence': 'I pick him up',
  'token': 502,
  'token_str': ' him'}]

In [ ]:
fill_mask("I <mask> to shopping")

[{'score': 0.21202704310417175,
  'sequence': 'I got to shopping',
  'token': 449,
  'token_str': ' got'},
 {'score': 0.12016503512859344,
  'sequence': 'I like to shopping',
  'token': 349,
  'token_str': ' like'},
 {'score': 0.0990045964717865,
  'sequence': 'I want to shopping',
  'token': 427,
  'token_str': ' want'},
 {'score': 0.053962916135787964,
  'sequence': 'I wanted to shopping',
  'token': 1370,
  'token_str': ' wanted'},
 {'score': 0.04086555913090706,
  'sequence': 'I went to shopping',
  'token': 775,
  'token_str': ' went'}]

In [ ]:
fill_mask("Paris is the <mask> of France")

[{'score': 0.04591355472803116,
  'sequence': 'Paris is the end of France',
  'token': 1197,
  'token_str': ' end'},
 {'score': 0.03489715978503227,
  'sequence': 'Paris is the kind of France',
  'token': 549,
  'token_str': ' kind'},
 {'score': 0.030713440850377083,
  'sequence': 'Paris is the picture of France',
  'token': 822,
  'token_str': ' picture'},
 {'score': 0.028830023482441902,
  'sequence': 'Paris is the rest of France',
  'token': 1634,
  'token_str': ' rest'},
 {'score': 0.02075226418673992,
  'sequence': 'Paris is the name of France',
  'token': 766,
  'token_str': ' name'}]

## another childes model by smeylan

In [ ]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer
model = AutoModelForMaskedLM.from_pretrained('smeylan/childes-bert')
tokenizer = AutoTokenizer.from_pretrained("smeylan/childes-bert")


In [ ]:
Smey_fill_mask = pipeline(
    'fill-mask',
    model = model,
    tokenizer = tokenizer
)

In [ ]:
Smey_fill_mask("I [MASK] you")

[{'score': 0.07764840126037598,
  'sequence': 'i did you',
  'token': 2106,
  'token_str': 'did'},
 {'score': 0.059528592973947525,
  'sequence': 'i am you',
  'token': 2572,
  'token_str': 'am'},
 {'score': 0.05386057496070862,
  'sequence': 'i know you',
  'token': 2113,
  'token_str': 'know'},
 {'score': 0.04900241270661354,
  'sequence': 'i love you',
  'token': 2293,
  'token_str': 'love'},
 {'score': 0.03938209265470505,
  'sequence': 'i see you',
  'token': 2156,
  'token_str': 'see'}]

In [ ]:
Smey_fill_mask("I pick [MASK] up")

[{'score': 0.5061348676681519,
  'sequence': 'i pick it up',
  'token': 2009,
  'token_str': 'it'},
 {'score': 0.16822730004787445,
  'sequence': 'i pick them up',
  'token': 2068,
  'token_str': 'them'},
 {'score': 0.0479658804833889,
  'sequence': 'i pick up up',
  'token': 2039,
  'token_str': 'up'},
 {'score': 0.04171311855316162,
  'sequence': 'i pick you up',
  'token': 2017,
  'token_str': 'you'},
 {'score': 0.035892508924007416,
  'sequence': 'i pick one up',
  'token': 2028,
  'token_str': 'one'}]

In [ ]:
Smey_fill_mask("I [MASK] to shopping")

[{'score': 0.20618851482868195,
  'sequence': 'i went to shopping',
  'token': 2253,
  'token_str': 'went'},
 {'score': 0.1555020958185196,
  'sequence': 'i go to shopping',
  'token': 2175,
  'token_str': 'go'},
 {'score': 0.11096654832363129,
  'sequence': 'i want to shopping',
  'token': 2215,
  'token_str': 'want'},
 {'score': 0.07313667982816696,
  'sequence': 'i going to shopping',
  'token': 2183,
  'token_str': 'going'},
 {'score': 0.04240037873387337,
  'sequence': 'i got to shopping',
  'token': 2288,
  'token_str': 'got'}]

In [ ]:
Smey_fill_mask("Paris is the [MASK] of France")

[{'score': 0.9964656829833984,
  'sequence': 'paris is the capital of france',
  'token': 3007,
  'token_str': 'capital'},
 {'score': 0.0015605653170496225,
  'sequence': 'paris is the city of france',
  'token': 2103,
  'token_str': 'city'},
 {'score': 0.0008385310065932572,
  'sequence': 'paris is the centre of france',
  'token': 2803,
  'token_str': 'centre'},
 {'score': 0.0002896166988648474,
  'sequence': 'paris is the center of france',
  'token': 2415,
  'token_str': 'center'},
 {'score': 0.0001177552476292476,
  'sequence': 'paris is the university of france',
  'token': 2118,
  'token_str': 'university'}]